In [29]:
import numpy as np
import pandas as pd
from scipy import stats
import random
import statsmodels.api as sm
songs=pd.read_csv('data/tracks.csv')
songs=songs.sample(100000)

In [30]:
songs_for_linear_regression = songs.copy()
songs_for_linear_regression.set_index('id',inplace=True)

In [31]:
songs_for_linear_regression.dropna(inplace=True)

In [32]:
songs_for_linear_regression

,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
id,,,,,,,,,,,,,,,,,,,
1YNCi9tPJffDJHNc6at129,Peaceful,38,173987,0,['Helen Reddy'],['0Sq7oGrYEe0BDmb13wgjOO'],1972,0.481,0.400,7,-11.236,1,0.0307,0.311000,0.000004,0.1120,0.442,139.614,4
1EKz4nd5HoV2IKZxOKaNA3,"Pelléas et Mélisande, L. 88 / Act 4: ""Pelléas ...",0,187000,0,"['Claude Debussy', 'Heinz Rehfuss', 'André Ves...","['1Uff91EOsvd99rtAupatMP', '3DBmYRD9rJmWOXR8Mk...",1952-09-03,0.380,0.125,1,-21.682,0,0.0690,0.951000,0.000007,0.1560,0.129,138.558,4
5uBU1KYDfb2qQMJJ0TZkc8,Let's Fall In Love,25,196333,0,"['Louis Armstrong', 'Oscar Peterson']","['19eLuQmk9aCobbVDHc6eek', '6zkX5fhrSD4tdVOmim...",1957-10-14,0.533,0.358,10,-10.961,1,0.1150,0.227000,0.000000,0.1170,0.543,134.882,4
5QgWYUWuhBJgocf6dQeu56,Sinfonia (Mixed),21,44280,0,['Mark Sixma'],['3ePCIHipMKD4n8IBXBYWSm'],2018-12-14,0.595,0.898,11,-5.409,1,0.0934,0.000005,0.317000,0.3270,0.286,130.008,4
2xQHS2PlKPK2WZ6Zz7xGng,You and the Night and the Music,24,155493,0,['Frank Sinatra'],['1Mxqyy3pSjf8kZZL4QVxS0'],1960-12-19,0.682,0.300,10,-11.720,0,0.0398,0.717000,0.000000,0.1770,0.605,128.760,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0NoGcqAFV5wzJfxyWXpvnP,Wish you were waiting,0,232956,0,['Mildred Bailey'],['0LyTHMstvqKiSoqdnnjqix'],1944,0.264,0.426,3,-5.663,1,0.0386,0.884000,0.000005,0.1640,0.240,76.075,4
1G93i9ga7ZPEOqpiNhM1mq,I Wanna Fuck You,63,178840,1,"['Snoop Dogg', 'Akon']","['7hJcb9fa4alzcOq3EaNPoG', '0z4gvV4rjIZ9wHck67...",2006-01-01,0.861,0.479,7,-8.433,1,0.0515,0.045200,0.000000,0.3510,0.278,99.969,4
4YSseVTVAOEbk0nVquoUrM,So Intense,20,290507,0,['Lisa Fischer'],['75NLTo547GvC8W5gmRDLLn'],1991-04-09,0.591,0.751,9,-10.534,1,0.0428,0.150000,0.000011,0.2080,0.765,80.190,4


In [33]:
def drop_columns(dataframe,to_be_deleted):
    dataframe.drop(to_be_deleted, axis=1, inplace=True)
to_be_deleted = ['id_artists', 'artists','name','release_date'] 

In [34]:
songs_for_linear_regression['release_date'] = pd.to_datetime(songs_for_linear_regression['release_date'], errors='coerce')

In [35]:
songs_for_linear_regression['year'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).year
songs_for_linear_regression['month'] = pd.DatetimeIndex(songs_for_linear_regression['release_date']).month

In [36]:
import sklearn
from sklearn import preprocessing

def scaler(dataframe,to_be_scaled):
    scaler = sklearn.preprocessing.StandardScaler(copy = True)
    dataframe[to_be_scaled] = scaler.fit_transform(dataframe[to_be_scaled].to_numpy())
# to_be_scaled = ['popularity','danceability','energy','key','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','time_signature']
to_be_scaled = ['loudness','duration_ms','tempo']

In [37]:
songs_for_linear_regression.columns

Index(['name', 'popularity', 'duration_ms', 'explicit', 'artists',
       'id_artists', 'release_date', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'time_signature', 'year', 'month'],
      dtype='object')

In [38]:
drop_columns(songs_for_linear_regression,to_be_deleted)#delete columns
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,year,month
id,,,,,,,,,,,,,,,,,
1YNCi9tPJffDJHNc6at129,38,173987,0,0.481,0.400,7,-11.236,1,0.0307,0.311000,0.000004,0.1120,0.442,139.614,4,1972,1
1EKz4nd5HoV2IKZxOKaNA3,0,187000,0,0.380,0.125,1,-21.682,0,0.0690,0.951000,0.000007,0.1560,0.129,138.558,4,1952,9
5uBU1KYDfb2qQMJJ0TZkc8,25,196333,0,0.533,0.358,10,-10.961,1,0.1150,0.227000,0.000000,0.1170,0.543,134.882,4,1957,10
5QgWYUWuhBJgocf6dQeu56,21,44280,0,0.595,0.898,11,-5.409,1,0.0934,0.000005,0.317000,0.3270,0.286,130.008,4,2018,12
2xQHS2PlKPK2WZ6Zz7xGng,24,155493,0,0.682,0.300,10,-11.720,0,0.0398,0.717000,0.000000,0.1770,0.605,128.760,4,1960,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0NoGcqAFV5wzJfxyWXpvnP,0,232956,0,0.264,0.426,3,-5.663,1,0.0386,0.884000,0.000005,0.1640,0.240,76.075,4,1944,1
1G93i9ga7ZPEOqpiNhM1mq,63,178840,1,0.861,0.479,7,-8.433,1,0.0515,0.045200,0.000000,0.3510,0.278,99.969,4,2006,1
4YSseVTVAOEbk0nVquoUrM,20,290507,0,0.591,0.751,9,-10.534,1,0.0428,0.150000,0.000011,0.2080,0.765,80.190,4,1991,4


In [40]:
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,time_signature,year,month
id,,,,,,,,,,,,,,,,
1YNCi9tPJffDJHNc6at129,38,173987,0,0.481,0.400,7,-11.236,1,0.0307,0.311000,0.000004,0.1120,139.614,4,1972,1
1EKz4nd5HoV2IKZxOKaNA3,0,187000,0,0.380,0.125,1,-21.682,0,0.0690,0.951000,0.000007,0.1560,138.558,4,1952,9
5uBU1KYDfb2qQMJJ0TZkc8,25,196333,0,0.533,0.358,10,-10.961,1,0.1150,0.227000,0.000000,0.1170,134.882,4,1957,10
5QgWYUWuhBJgocf6dQeu56,21,44280,0,0.595,0.898,11,-5.409,1,0.0934,0.000005,0.317000,0.3270,130.008,4,2018,12
2xQHS2PlKPK2WZ6Zz7xGng,24,155493,0,0.682,0.300,10,-11.720,0,0.0398,0.717000,0.000000,0.1770,128.760,4,1960,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0NoGcqAFV5wzJfxyWXpvnP,0,232956,0,0.264,0.426,3,-5.663,1,0.0386,0.884000,0.000005,0.1640,76.075,4,1944,1
1G93i9ga7ZPEOqpiNhM1mq,63,178840,1,0.861,0.479,7,-8.433,1,0.0515,0.045200,0.000000,0.3510,99.969,4,2006,1
4YSseVTVAOEbk0nVquoUrM,20,290507,0,0.591,0.751,9,-10.534,1,0.0428,0.150000,0.000011,0.2080,80.190,4,1991,4


In [41]:
def scale_whole_df_except_valence():
    valence_only = songs_for_linear_regression['valence']
    del songs_for_linear_regression["valence"]

    x = songs_for_linear_regression #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    songs_for_linear_regression[songs_for_linear_regression.columns] = min_max_scaler.fit_transform(songs_for_linear_regression[songs_for_linear_regression.columns].to_numpy())
    songs_for_linear_regression['valence'] = valence_only
scale_whole_df_except_valence()

In [42]:
songs_for_linear_regression

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,tempo,time_signature,year,month,valence
id,,,,,,,,,,,,,,,,,
1YNCi9tPJffDJHNc6at129,0.383838,0.033866,0.0,0.487335,0.400,0.636364,0.755048,1.0,0.031649,0.312249,0.000004,0.1120,0.573347,0.8,0.505051,0.000000,0.442
1EKz4nd5HoV2IKZxOKaNA3,0.000000,0.036448,0.0,0.385005,0.125,0.090909,0.593305,0.0,0.071134,0.954819,0.000007,0.1560,0.569010,0.8,0.303030,0.727273,0.129
5uBU1KYDfb2qQMJJ0TZkc8,0.252525,0.038300,0.0,0.540020,0.358,0.909091,0.759306,1.0,0.118557,0.227912,0.000000,0.1170,0.553914,0.8,0.353535,0.818182,0.543
5QgWYUWuhBJgocf6dQeu56,0.212121,0.008124,0.0,0.602837,0.898,1.000000,0.845271,1.0,0.096289,0.000005,0.317000,0.3270,0.533898,0.8,0.969697,1.000000,0.286
2xQHS2PlKPK2WZ6Zz7xGng,0.242424,0.030195,0.0,0.690983,0.300,0.909091,0.747554,0.0,0.041031,0.719880,0.000000,0.1770,0.528773,0.8,0.383838,1.000000,0.605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0NoGcqAFV5wzJfxyWXpvnP,0.000000,0.045568,0.0,0.267477,0.426,0.272727,0.841338,1.0,0.039794,0.887550,0.000005,0.1640,0.312414,0.8,0.222222,0.000000,0.240
1G93i9ga7ZPEOqpiNhM1mq,0.636364,0.034829,1.0,0.872340,0.479,0.636364,0.798449,1.0,0.053093,0.045382,0.000000,0.3510,0.410539,0.8,0.848485,0.000000,0.278
4YSseVTVAOEbk0nVquoUrM,0.202020,0.056990,0.0,0.598784,0.751,0.818182,0.765917,1.0,0.044124,0.150602,0.000011,0.2080,0.329313,0.8,0.696970,0.272727,0.765


In [43]:
songs_for_linear_regression["key"] = songs_for_linear_regression["key"].astype("category")
songs_for_linear_regression = pd.get_dummies(songs_for_linear_regression, columns=["key"])

* As always when coverting to dummy variables, we will rename them to logical names. 
* In the following loop we assign rename a column to a key_x where x is the number accordingly

In [44]:
starting_key = 0 
for i in range (16, 28):
    songs_for_linear_regression = songs_for_linear_regression.rename(columns={songs_for_linear_regression.columns[i]: 'key_%d' % starting_key})
    starting_key = starting_key  + 1 

In [45]:
songs_for_linear_regression.head(5)

,popularity,duration_ms,explicit,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,...,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
id,,,,,,,,,,,,,,,,,,,,,
1YNCi9tPJffDJHNc6at129,0.383838,0.033866,0.0,0.487335,0.400,0.755048,1.0,0.031649,0.312249,0.000004,...,0,0,0,0,0,1,0,0,0,0
1EKz4nd5HoV2IKZxOKaNA3,0.000000,0.036448,0.0,0.385005,0.125,0.593305,0.0,0.071134,0.954819,0.000007,...,0,0,0,0,0,0,0,0,0,0
5uBU1KYDfb2qQMJJ0TZkc8,0.252525,0.038300,0.0,0.540020,0.358,0.759306,1.0,0.118557,0.227912,0.000000,...,0,0,0,0,0,0,0,0,1,0
5QgWYUWuhBJgocf6dQeu56,0.212121,0.008124,0.0,0.602837,0.898,0.845271,1.0,0.096289,0.000005,0.317000,...,0,0,0,0,0,0,0,0,0,1
2xQHS2PlKPK2WZ6Zz7xGng,0.242424,0.030195,0.0,0.690983,0.300,0.747554,0.0,0.041031,0.719880,0.000000,...,0,0,0,0,0,0,0,0,1,0


In [46]:
# scaler(songs_for_linear_regression,to_be_scaled)

In [47]:
songs_for_linear_regression.columns

Index(['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'tempo', 'time_signature', 'year', 'month', 'valence',
       'key_0', 'key_1', 'key_2', 'key_3', 'key_4', 'key_5', 'key_6', 'key_7',
       'key_8', 'key_9', 'key_10', 'key_11'],
      dtype='object')

We move our dependent variable to the last column

In [48]:
x = songs_for_linear_regression['valence']
del songs_for_linear_regression["valence"]
songs_for_linear_regression['valence'] = x

In [49]:
songs_for_linear_regression.columns[-1]

'valence'

# AND SO IT BEGINS

# CODE FOR FORWARD SELECTION

In [50]:
def process_subset(y, data, feature_set):
    X = data.loc[:, feature_set].values
    X = sm.add_constant(X)
    names = ['intercept']
    names.extend(feature_set)
    model = sm.OLS(y, X)
    model.data.xnames = names
    regr = model.fit()
    return regr

In [51]:
import itertools

def get_best_of_k(y, data, k):
    
    best_rsquared = 0
    best_model = None
    for comb in itertools.combinations(data.columns, k):
        regr = process_subset(y, data, comb)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr

    return best_model

In [52]:
def best_subset_selection(data, exog):
    best_model = None
    best_models = []
    y = data.loc[:, exog]
    endog = [ x for x in data.columns if x != exog ]
    X = data.loc[:, endog]

    for i in range(1, len(data.columns)):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model = get_best_of_k(y, X, i)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(model.model.data.xnames[1:]) # get the variables minums the intercept
        best_models.append(model)

    print(f'Fitted {2**len(data.columns)} models')
    return best_model, best_models

In [53]:
def forward_add_variable(data, exog, selected, to_select):
    best_rsquared = 0
    best_model = None
    best_column = None
    y = data.loc[:, exog]
    
    for column in to_select:
        new_selected = selected + [column]
        regr = process_subset(y, data, new_selected)
        if regr.rsquared > best_rsquared:
            best_rsquared = regr.rsquared
            best_model = regr
            best_column = column
    
    return best_model, best_column

In [54]:
def forward_stepwise_selection(data, exog):

    best_models = []
    best_model = None
    selected = []
    to_select = [ x for x in data.columns if x != exog ]

    p = len(to_select) + 1

    for i in range(1, p):
        print(f'Finding the best model for {i} variable{"s" if i > 1 else ""}')
        model, best_column = forward_add_variable(data, exog, selected, to_select)
        selected.append(best_column)
        to_select.remove(best_column)
        if not best_model or model.rsquared_adj > best_model.rsquared_adj:
            best_model = model
        print(selected)
        best_models.append(model)
        print(model.rsquared_adj)
        
    print(f'Fitted {1 + p*(p+1)//2} models')
    return best_model, best_models

In [55]:
best_model, _ = forward_stepwise_selection(songs_for_linear_regression, 'valence')
print('Best overall model:', len(best_model.model.exog_names), best_model.model.exog_names)

Finding the best model for 1 variable
['danceability']
0.3120484046776547
Finding the best model for 2 variables
['danceability', 'energy']
0.3693229621412223
Finding the best model for 3 variables
['danceability', 'energy', 'year']
0.45308862179668463
Finding the best model for 4 variables
['danceability', 'energy', 'year', 'tempo']
0.4655501654930835
Finding the best model for 5 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms']
0.477060925298305
Finding the best model for 6 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness']
0.4854412909503084
Finding the best model for 7 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness', 'explicit']
0.49058291865387493
Finding the best model for 8 variables
['danceability', 'energy', 'year', 'tempo', 'duration_ms', 'speechiness', 'explicit', 'acousticness']
0.4952835591340258
Finding the best model for 9 variables
['danceability', 'energy', 'year', 'tempo', 'duration

In [56]:
best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                valence   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.506
Method:                 Least Squares   F-statistic:                     4263.
Date:                Tue, 22 Feb 2022   Prob (F-statistic):               0.00
Time:                        19:58:35   Log-Likelihood:                 27092.
No. Observations:              100000   AIC:                        -5.413e+04
Df Residuals:                   99975   BIC:                        -5.390e+04
Df Model:                          24                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
intercept            0.1343      0.010     14.043      0.000       0.116       0.153
danceability         0.8248      0.004    215.378      0.000       0.817       0.832
energy               0.5595      0.005    119.467      0.000       0.550       0.569
year                -0.2824      0.003    -88.039      0.000      -0.289      -0.276
tempo                0.2386      0.005     48.512      0.000       0.229       0.248
duration_ms         -0.9886      0.022    -45.864      0.000      -1.031      -0.946
speechiness         -0.1425      0.004    -40.437      0.000      -0.149      -0.136
explicit            -0.0923      0.003    -28.466      0.000      -0.099      -0.086
acousticness         0.1060      0.003     38.502      0.000       0.101       0.111
instrumentalness    -0.0650      0.002    -31.194      0.000      -0.069      -0.061
loudness            -0.3805      0.012    -32.013      0.000      -0.404      -0.357
mode                 0.0149      0.001     11.413      0.000       0.012       0.017
key_1               -0.0197      0.002     -8.156      0.000      -0.024      -0.015
popularity           0.0219      0.003      6.645      0.000       0.015       0.028
key_5                0.0062      0.002      2.907      0.004       0.002       0.010
key_6               -0.0164      0.003     -5.811      0.000      -0.022      -0.011
key_8               -0.0121      0.003     -4.737      0.000      -0.017      -0.007
month               -0.0053      0.002     -3.157      0.002      -0.009      -0.002
liveness             0.0109      0.003      3.158      0.002       0.004       0.018
key_3               -0.0113      0.003     -3.780      0.000      -0.017      -0.005
key_4               -0.0089      0.002     -3.727      0.000      -0.014      -0.004
key_11              -0.0088      0.003     -3.309      0.001      -0.014      -0.004
time_signature       0.0163      0.006      2.718      0.007       0.005       0.028
key_2               -0.0038      0.002     -1.801      0.072      -0.008       0.000
key_0               -0.0021      0.002     -1.068      0.286      -0.006       0.002
==============================================================================
Omnibus:                      328.235   Durbin-Watson:                   1.989
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              331.484
Skew:                          -0.140   Prob(JB):                     1.04e-72
Kurtosis:                       3.039   Cond. No.                         77.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""